In [4]:
%%capture
import numpy as np
import pandas as pd
import scipy.stats as st
import itertools
import math
from collections import Counter, defaultdict, OrderedDict
%load_ext autoreload
%autoreload 2

import cv2
import pprint
import pickle
import json
import requests
import io

import pdfextraction.ocr_pipeline

In [16]:
import json
import pprint

In [4]:
test_book = book_list[8]
# pdfextraction.ocr_pipeline.process_book(test_book, 0, word_margin=0.1, line_overlap=0.5, char_margin=2.0,
#                         line_margin=0.5, boxes_flow=0.5)

In [89]:
bb = make_page_breakdowns()

('Spectrum_Science,: ', 6, ' total')
('Daily_Science: ', 6, ' total')
('Read_and_Understand Science: ', 4, ' total')
('workbooks: ', 5, ' total')


In [6]:
book_breakdowns['spectrum_sci']

['Spectrum_Science_Grade_3.pdf',
 'Spectrum_Science_Grade_4.pdf',
 'Spectrum_Science_Grade_5.pdf',
 'Spectrum_Science_Grade_6.pdf',
 'Spectrum_Science_Grade_7.pdf',
 'Spectrum_Science_Grade_8.pdf']

In [76]:
pdfextraction.ocr_pipeline.process_book(book_breakdowns['daily_sci'][5], [120,120], word_margin=0.1, line_overlap=0.5, char_margin=2.0,
                    line_margin=0.5, boxes_flow=0.5)

In [35]:
for book in book_breakdowns['daily_sci']:
    print(book)
    pdfextraction.ocr_pipeline.process_book(book, 0, word_margin=0.1, line_overlap=0.5, char_margin=2.0,
                        line_margin=0.5, boxes_flow=0.5)

Daily_Science_Grade_1_Evan_Moor.pdf
Daily_Science_Grade_2_Evan_Moor.pdf
Daily_Science_Grade_3_(Daily_Practice_Books)_Evan_Moore.pdf
Daily_Science_Grade_4_Evan_Moor.pdf
Daily_Science_Grade_5_Evan_Moor.pdf
Daily_Science_Grade_6_(Daily_Practice_Books)_Evan_Moor.pdf


In [95]:
import pickle

In [97]:
with open('breakdowns.pkl', 'w') as f:
        pickle.dump(book_breakdowns, f)

In [6]:
with open('breakdowns.pkl', 'r') as f:
    book_breakdowns = pickle.load(f)

In [7]:
tbt = book_breakdowns['daily_sci'][5]
tbt

'Daily_Science_Grade_6_(Daily_Practice_Books)_Evan_Moor.pdf'

In [101]:
book_breakdowns

defaultdict(list,
            {'daily_sci': ['Daily_Science_Grade_1_Evan_Moor.pdf',
              'Daily_Science_Grade_2_Evan_Moor.pdf',
              'Daily_Science_Grade_3_(Daily_Practice_Books)_Evan_Moore.pdf',
              'Daily_Science_Grade_4_Evan_Moor.pdf',
              'Daily_Science_Grade_5_Evan_Moor.pdf',
              'Daily_Science_Grade_6_(Daily_Practice_Books)_Evan_Moor.pdf'],
             'misc': ['Barron\xe2\x80\x99s_New_York_State_Grade_4_Elementary-Level_Science_Test.pdf',
              'Confusing_Science_Terms_Grades_5_-_8_Schyrlet_Cameron.pdf',
              'General_Science_Grades_5_-_8_(Daily_Skill_Builders).pdf',
              'STAAR_Success_Strategies_Grade_5_Science_Study_Guide.pdf',
              'Science_Essentials_Grades_3-4_School_Specialty_Publishing.pdf',
              'Steck-Vaughn_GED_Test_Preparation_Student_Edition_Science_2014.pdf',
              'The_New_Children\xe2\x80\x99s_Encyclopedia_DK_Publishing.pdf'],
             'read_und_sci': ['Read_a

In [86]:
pdfextraction.ocr_pipeline.perform_ocr(tbt, 'annotations', [40, 41])

('Daily_Science_Grade_1_Evan_Moor', 40)
('Daily_Science_Grade_1_Evan_Moor', 41)


In [51]:
page_ranges = pd.read_csv('./pdfs/page_ranges.csv', delimiter=' ')
page_ranges.head(1)

,book_name,start_page,end_page
0,Barron’s_New_York_State_Grade_4_Elementary-Lev...,11,281


In [2]:
with open('pdfs/page_ranges.csv') as f:
    ranges = f.readlines()
range_lookup = {line.split(' ')[0]:[int(num) for num in line.strip().split(' ')[1:]] for line in ranges}

In [85]:
range_lookup[tbt]

[8, 187]

In [106]:
with open('pdfs/page_ranges.csv') as f:
    ranges = f.readlines()
range_lookup = {line.split(' ')[0]:[int(num) for num in line.strip().split(' ')[1:]] for line in ranges}

In [8]:
range_lookup['Daily_Science_Grade_3_(Daily_Practice_Books)_Evan_Moore.pdf']

[85, 194]

In [108]:
range_lookup

{'Barron\xe2\x80\x99s_New_York_State_Grade_4_Elementary-Level_Science_Test.pdf': [11,
  281],
 'Confusing_Science_Terms_Grades_5_-_8_Schyrlet_Cameron.pdf': [9, 76],
 'DK_Workbooks_Science_Second_Grade_DK_Publishing.pdf': [9, 64],
 'DK_Workbooks_Science_Third_Grade_DK_Publishing.pdf': [8, 64],
 'Daily_Science_Grade_1_Evan_Moor.pdf': [8, 187],
 'Daily_Science_Grade_2_Evan_Moor.pdf': [8, 192],
 'Daily_Science_Grade_3_(Daily_Practice_Books)_Evan_Moore.pd': [8, 194],
 'Daily_Science_Grade_4_Evan_Moor.pdf': [8, 194],
 'Daily_Science_Grade_5_Evan_Moor.pdf': [8, 195],
 'Daily_Science_Grade_6_(Daily_Practice_Books)_Evan_Moor.pdf': [8, 195],
 'GED_Science_Exam_Secrets_Workbook_GED_Test_Practice_Questions.pdf': [6, 113],
 'General_Science_Grades_5_-_8_(Daily_Skill_Builders).pdf': [8, 98],
 'Read_and_Understand_Science_Grades_1-2.pdf': [6, 139],
 'Read_and_Understand_Science_Grades_2-3.pdf': [4, 146],
 'Read_and_Understand_Science_Grades_3-4.pdf': [5, 146],
 'Read_and_Understand_Science_Grades_4-6

In [11]:
def query_vision_ocr(image_url, merge_boxes=True, include_merged_components=True, as_json=True):
    api_entry_point = 'http://vision-ocr.dev.allenai.org/v1/ocr'
    header = {'Content-Type': 'application/json'}
    request_data = {
        'url': image_url,
        # 'maximumSizePixels': max_pix_size,
        'mergeBoxes': merge_boxes,
        'includeMergedComponents': include_merged_components
    }

    json_data = json.dumps(request_data)
    response = requests.post(api_entry_point, data=json_data, headers=header)
    print(response.reason)
    json_response = json.loads(response.content.decode())
    if as_json:
        response = json_response
    return response

def form_image_url(book_name, page_n, ):
    book_name_no_ext = book_name.replace('.pdf', '_')
    image_base = 'https://s3-us-west-2.amazonaws.com/ai2-vision-turk-data/textbook-annotation-test/page-images/'
    return image_base + book_name_no_ext + str(page_n) + '.jpeg'

In [14]:
test_page = form_image_url(book_breakdowns['daily_sci'][3], 55)
test_page

'https://s3-us-west-2.amazonaws.com/ai2-vision-turk-data/textbook-annotation-test/page-images/Daily_Science_Grade_4_Evan_Moor_55.jpeg'

In [15]:
anno_respone = query_vision_ocr(test_page)

OK


In [18]:
pprint.pprint(anno_respone)

{u'detections': [{u'rectangle': [{u'x': 1973, u'y': 138},
                                 {u'x': 2251, u'y': 464}],
                  u'score': 0.75,
                  u'value': u'scieoe Big Idea 2'},
                 {u'rectangle': [{u'x': 2097, u'y': 138},
                                 {u'x': 2251, u'y': 212}],
                  u'score': 0.5,
                  u'value': u'scieoe'},
                 {u'rectangle': [{u'x': 1973, u'y': 268},
                                 {u'x': 2224, u'y': 409}],
                  u'score': 0.5,
                  u'value': u'Big'},
                 {u'rectangle': [{u'x': 1983, u'y': 410},
                                 {u'x': 2213, u'y': 464}],
                  u'score': 0.5,
                  u'value': u'Idea 2'},
                 {u'rectangle': [{u'x': 140, u'y': 215},
                                 {u'x': 252, u'y': 260}],
                  u'score': 1.0,
                  u'value': u'Name'},
                 {u'rectangle': [{u'x': 194, 